In [48]:
# pytorch - 3

# creating simple NN manually

In [49]:
import numpy as np
import pandas as pd
import torch

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder

In [50]:
# data loading
# breast cancer

url = 'https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/master/data.csv'
df = pd.read_csv(url)

df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [51]:
# basic EDA

print("nulls ? :",df.isnull().sum())      # checking for NULLS

print("duplicates ?",df.duplicated().sum())  # checking for duplicates

print("Basic info :",df.info())

print("shape :",df.shape)

nulls ? : id                           0
diagnosis                    0
radius_mean                  0
texture_mean                 0
perimeter_mean               0
area_mean                    0
smoothness_mean              0
compactness_mean             0
concavity_mean               0
concave points_mean          0
symmetry_mean                0
fractal_dimension_mean       0
radius_se                    0
texture_se                   0
perimeter_se                 0
area_se                      0
smoothness_se                0
compactness_se               0
concavity_se                 0
concave points_se            0
symmetry_se                  0
fractal_dimension_se         0
radius_worst                 0
texture_worst                0
perimeter_worst              0
area_worst                   0
smoothness_worst             0
compactness_worst            0
concavity_worst              0
concave points_worst         0
symmetry_worst               0
fractal_dimension_worst      

In [52]:
# dropping unwanted columns

df.drop(['id','Unnamed: 32'],axis=1,inplace=True)

In [53]:
# defining features & target

X = df.drop('diagnosis',axis=1)
y = df['diagnosis']

In [54]:
# classes count

counts = df['diagnosis'].value_counts()

print(counts)

percentage = np.round((counts / len(df)) * 100,2)

print(percentage)

diagnosis
B    357
M    212
Name: count, dtype: int64
diagnosis
B    62.74
M    37.26
Name: count, dtype: float64


In [55]:
# encoding target

le = LabelEncoder()
y = le.fit_transform(y)

# B : Non-Cancerous : 0
# M : Cancerous     : 1

In [56]:
# train-test split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=38)

In [57]:
# scaling

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [58]:
# to tensors

X_train_t = torch.from_numpy(X_train)
X_test_t = torch.from_numpy(X_test)

y_train_t = torch.from_numpy(y_train)
y_test_t = torch.from_numpy(y_test)

In [59]:
print(X_train_t.shape)
print(X_test_t.shape)

print(y_train_t.shape)
print(y_test_t.shape)

torch.Size([455, 30])
torch.Size([114, 30])
torch.Size([455])
torch.Size([114])


In [60]:
# defining simple NN

class SimpleNN():

  def __init__(self,X):          # constructor
    self.weights = torch.rand(X.shape[1],1,dtype=torch.float64,requires_grad=True)
    self.bias = torch.zeros(1,dtype=torch.float64,requires_grad=True)

    # X.shape[1] = 30 , since n_weights = n_feats
    # bias = 1 , since n_output = n_bias

    # torch.randn() : n = normal distribution , (-inf , inf) ; to tackel VANISHING grad

    # requires_grad=True ; grad tracking

  def forward(self,X):
    z = torch.matmul(X,self.weights) + self.bias
    y_pred = torch.sigmoid(z)
    return y_pred

    # z = w.X + b
    # y_pred = sigmoid(z)

  def loss_fn(self,y_pred,y):
    epsilon = 1e-7
    y_pred = torch.clamp(y_pred,epsilon,1-epsilon)      # to avoid log(0) = -inf trap

    loss = -(y_train_t * torch.log(y_pred) + (1-y_train_t) * torch.log(1-y_pred)).mean()     # Binary Cross Entropy formula
    return loss                                                                              # mean() = one output ; avg of all loss


In [61]:
learning_rate = 0.1

epochs = 25

In [62]:
# training pipeline

model = SimpleNN(X_train_t)

for epoch in range(epochs):
  y_pred = model.forward(X_train_t)           # forward pass

  loss = model.loss_fn(y_pred,y_train_t)      # loss calculation

  loss.backward()                             # back propagation

  with torch.no_grad():                       # params updates
    model.weights -= learning_rate * model.weights.grad
    model.bias -= learning_rate * model.bias.grad

    # w(new) = w(old) - lr (dL/dw)
    # b(new) = b(old) - lr (dL/db)

  model.weights.grad.zero_()                  # NO accumalation
  model.bias.grad.zero_()                     # NO accumalation

  print(f"Epoch: {epoch + 1} Loss: {loss.item()}")

Epoch: 1 Loss: 3.727631579473536
Epoch: 2 Loss: 3.595707443086786
Epoch: 3 Loss: 3.461620793275099
Epoch: 4 Loss: 3.3217995321830984
Epoch: 5 Loss: 3.1725465147795022
Epoch: 6 Loss: 3.020748053294865
Epoch: 7 Loss: 2.8656911790062325
Epoch: 8 Loss: 2.70718444396773
Epoch: 9 Loss: 2.5477449776579473
Epoch: 10 Loss: 2.3806429788598904
Epoch: 11 Loss: 2.212917387106273
Epoch: 12 Loss: 2.0450591059000103
Epoch: 13 Loss: 1.8801861779230353
Epoch: 14 Loss: 1.718060245651586
Epoch: 15 Loss: 1.559826572343393
Epoch: 16 Loss: 1.414178626261394
Epoch: 17 Loss: 1.2830373747409587
Epoch: 18 Loss: 1.1683003924331015
Epoch: 19 Loss: 1.0714863794726814
Epoch: 20 Loss: 0.9931839455471185
Epoch: 21 Loss: 0.9325593254982129
Epoch: 22 Loss: 0.887357129187237
Epoch: 23 Loss: 0.8544558534568846
Epoch: 24 Loss: 0.8305986794898652
Epoch: 25 Loss: 0.8129650775920237


In [64]:
# evaluation

with torch.no_grad():
  y_pred = model.forward(X_test_t)
  y_pred = (y_pred > 0.5).float()

  accuracy = (y_pred == y_test_t).float().mean()

  print(f'{accuracy:.2f}')


0.53
